In [1]:
import pandas as pd
import numpy as np

### Non-contextual bandit problem (trivial state).

Consider the sub-problem where rewards depend on action but not state. In this case there is a single best action for everyone (no personalization opportunity).

You're running a website for an artist who has four products. You're making a popup with a special offer to sign up for a mailing list, which offer should you choose?

In [2]:
def get_noncon_reward(action):
    if action == 'a0':
        return np.random.binomial(1, 0.03)
    elif action == 'a1':
        return np.random.binomial(1, 0.02)
    elif action == 'a2' or action == 'a3':
        return np.random.binomial(1, 0.01)
    else:
        raise ValueError('Unrecognized action.')

In [3]:
num_sessions = 100000

#### Try number 1 - random serving:

In [4]:
rewards = []
action_cnt_dict = {'a0': 0, 'a1': 0, 'a2': 0, 'a3': 0}

for i in range(num_sessions):
    action = np.random.choice(['a0', 'a1', 'a2', 'a3'])
    action_cnt_dict[action] += 1
    rewards.append(get_noncon_reward(action))
    
cumulative_reward = np.array(rewards).sum()
print('Profit: ', cumulative_reward)
print(action_cnt_dict)

Profit:  1762
{'a0': 24921, 'a1': 25077, 'a2': 25198, 'a3': 24804}


#### Try number 2 - explore first:

In [5]:
exploration_budget = num_sessions/4
learned = False
rewards = []
history = {'a0': [], 'a1': [], 'a2': [], 'a3': []}
action_cnt_dict = {'a0': 0, 'a1': 0, 'a2': 0, 'a3': 0}

for i in range(num_sessions):
    if (i < exploration_budget): #explore here!
        action = np.random.choice(['a0', 'a1', 'a2', 'a3'])
        action_cnt_dict[action] += 1
        reward = get_noncon_reward(action)
        rewards.append(reward)
        # add to history
        history[action].append(reward)
    else:
        if not learned:
            #learn!
            best_rate = 0.0
            best_action = 'unknown'
            for a in history:
                rate = np.array(history[a]).mean()
                if rate > best_rate:
                    best_rate = rate
                    best_action = a
            learned = True
            rewards.append(get_noncon_reward(best_action))
            action_cnt_dict[best_action] += 1
        else:
            #be greedy!
            rewards.append(get_noncon_reward(best_action))
            action_cnt_dict[best_action] += 1

            
cumulative_reward = np.array(rewards).sum()
print('Profit: ', cumulative_reward)
print('Best learned action: ', best_action)
print(action_cnt_dict)

Profit:  2664
Best learned action:  a0
{'a0': 81202, 'a1': 6330, 'a2': 6227, 'a3': 6241}


#### Thompson Sampling (simple but optimal bandit algorithm)

In [6]:
rewards = []
successes = [1,1,1,1]
failures = [1,1,1,1]
action_dict = {0: 'a0', 1: 'a1', 2: 'a2', 3: 'a3'}
action_cnt_dict = {'a0': 0, 'a1': 0, 'a2': 0, 'a3': 0}

for i in range(num_sessions):
    beta_samples = np.random.beta(successes, failures)
    action_index = np.argmax(beta_samples)
    action = action_dict[action_index]
    action_cnt_dict[action] += 1
    reward = get_noncon_reward(action)
    rewards.append(reward)
    #learn
    successes[action_index] += reward
    failures[action_index] += 1-reward
    
    
cumulative_reward = np.array(rewards).sum()
print('Profit: ', cumulative_reward)
print(action_cnt_dict)

Profit:  2896
{'a0': 95494, 'a1': 3198, 'a2': 334, 'a3': 974}
